# Hands-On Data Analysis with DuckDB

## Technical requirements

In order to run the examples in this notebook, you'll need to install the Python dependencies for this project. You can do this by running the following command in your terminal when in the root directory of the project. Note that ideally this should be using a Python virtual environment for this project.

    pip install -r requirements.txt

For complete instructions on how to set up your environment for working through the examples, please consult the *Technical requirements* section of the chapter *Setting up the DuckDB Python Client*.

## Loading the pedestrian counts dataset into DuckDB

### Establishing the data processing steps

In [ ]:
import duckdb

records = duckdb.read_csv("pedestrian_records_2009-2022.csv") 

records.show(max_width=200) 

In [ ]:
records = duckdb.read_csv( 
    "pedestrian_records_2009-2022.csv", 
    dtype={"Date_Time": "TIMESTAMP"}, 
    timestamp_format="%B %d, %Y %H:%M:%S %p", 
)

In [ ]:
records.limit(5).show(max_width=200)

In [ ]:
transformed = records.select("* EXCLUDE ID").sort("Date_Time")

### Loading the cleaned and transformed data into DuckDB

In [ ]:
with duckdb.connect("pedestrian.duckdb") as conn:
    result = (
        conn.read_csv(
            "pedestrian_records_2009-2022.csv",
            dtype={"Date_Time": "TIMESTAMP"},
            timestamp_format="%B %d, %Y %H:%M:%S %p",
        )
        .select("* EXCLUDE ID")
        .sort("Date_Time")
    )
    result.to_table("pedestrian_counts")

## Effective data analysis using Jupyter Notebooks 

## Convenient SQL queries with JupySQL

In [ ]:
conn = duckdb.connect("pedestrian.duckdb")

conn.sql(
    """
    SELECT sum(Hourly_Counts)
    FROM pedestrian_counts
    WHERE Year=2022 AND Sensor_Name='Melbourne Central'
    GROUP BY ALL
    """
)

In [ ]:
# configure JupySQL to use the default DuckDB database
%load_ext sql 
conn = duckdb.connect() 
%sql conn --alias duckdb 

In [ ]:
# for this exercise, we'll configure JupySQL to use a connection to the on-disk database
conn = duckdb.connect("pedestrian.duckdb")
%sql conn --alias pedestrian.duckdb

In [ ]:
# configure JupySQL to return Pandas dataframes by default
%config SqlMagic.autopandas = True 

In [ ]:
%%sql 
SELECT sum(Hourly_Counts) 
FROM pedestrian_counts 
WHERE Year=2022 AND Sensor_Name='Melbourne Central' 
GROUP BY ALL

In [ ]:
%%sql sensors_2022_df << 
SELECT Sensor_Name, sum(Hourly_Counts) as Total_Counts 
FROM pedestrian_counts 
WHERE Year=2022 
GROUP BY Sensor_Name 
ORDER BY Total_Counts DESC 

In [ ]:
sensors_2022_df.head(10)

### Interactive visualisations with Plotly

In [ ]:
import plotly.express as px

In [ ]:
figure = px.bar( 
    sensors_2022_df.head(10),
    x="Sensor_Name",
    y="Total_Counts",
    height=500,
    title="Top 10 sensors by pedestrian counts in 2022",
)

figure

In [ ]:
%%sql sensor_years_df << 
SELECT Year, COUNT(DISTINCT Sensor_Name) as Total_Sensors 
FROM pedestrian_counts 
GROUP BY Year 
ORDER BY Year 

In [ ]:
sensor_years_df.head(5)

In [ ]:
figure = px.line( 
    sensor_years_df, 
    x="Year",
    y="Total_Sensors",
    markers=True,
    height=500,
    title="Total number of active sensors by year"
)
figure

In [ ]:
figure.update_layout(xaxis={"dtick": 1}, title={"x": 0.5})

In [ ]:
figure.update_layout(xaxis_dtick=1, title_x=0.5)

## Analysing pedestrian traffic through Melbourne CBD

In [ ]:
%%sql year_counts_df <<
SELECT Year, sum(Hourly_Counts) AS Total_Counts
FROM pedestrian_counts
GROUP BY Year
ORDER BY Year

## Calculating Total pedestrian counts over time

In [ ]:
%%sql year_counts_df <<
SELECT Year, sum(Hourly_Counts) AS Total_Counts
FROM pedestrian_counts
GROUP BY Year
ORDER BY Year

In [ ]:
px.line(
    year_counts_df,
    x="Year",
    y="Total_Counts",
    markers=True,
    title="Total pedestrian counts by year",
)

In [ ]:
%%sql
CREATE OR REPLACE TABLE common_sensors AS
SELECT Sensor_Name FROM (
    PIVOT pedestrian_counts
    ON Year
    USING any_value(Hourly_Counts)
    GROUP BY Sensor_Name
)
WHERE COLUMNS(*) IS NOT NULL

In [ ]:
%sql SELECT COUNT(*) FROM common_sensors

In [ ]:
conn.table("common_sensors").count("*")

In [ ]:
%%sql year_counts_filtered_df <<
SELECT Year, sum(Hourly_Counts) AS Total_Counts
FROM pedestrian_counts
WHERE Sensor_Name IN (FROM common_sensors)
GROUP BY Year
ORDER BY Year

In [ ]:
px.line(
    year_counts_filtered_df,
    x="Year",
    y="Total_Counts",
    markers=True,
    title="Total pedestrian counts by year for sensors active across all years",
)

In [ ]:
%%sql year_month_counts_df << 
SELECT
    Year,
    Month,
    month(Date_Time) AS Month_Num,
    sum(Hourly_Counts) AS Total_Counts
FROM pedestrian_counts
WHERE Year IN (2019, 2020, 2021)
    AND Sensor_Name in (FROM common_sensors)
GROUP BY Year, Month, Month_Num
ORDER BY Year, Month_Num

In [ ]:
year_month_counts_df.head(15)

In [ ]:
px.line(
    year_month_counts_df,
    x="Month",
    y="Total_Counts",
    markers=True, 
    symbol="Year",
    symbol_sequence=["square", "diamond", "circle"],
    color="Year",
    title="Traffic by month for sensors active across 2019, 2020, and 2021",
).update_traces(marker_size=8)

## Timeseries plots of sensors

In [ ]:
%%sql sensor_2020_df << 
SELECT Hourly_Counts, Date_Time 
FROM pedestrian_counts 
WHERE Sensor_Name = 'Flinders La-Swanston St (West)'
    AND Year = 2020

In [ ]:
px.line(
    sensor_2020_df,
    y="Hourly_Counts",
    x="Date_Time",
    title="Hourly pedestrian traffic for Flinders La-Swanston St (West), 2020",
)

In [ ]:
%%sql multi_sensor_df <<
SELECT Sensor_Name, Hourly_Counts, Date_Time
FROM pedestrian_counts
WHERE Sensor_Name IN (
        'Town Hall (West)',
        'Bourke Street Mall (North)',
        'Southern Cross Station'
    )
    AND Year = 2019
    AND Month = 'September'

In [ ]:
px.line(
    multi_sensor_df,
    y="Hourly_Counts",
    x="Date_Time",
    facet_col="Sensor_Name",
    facet_col_wrap=1,
    title="Hourly pedestrian traffic for December 2019",
    height=600,
).update_layout(yaxis_fixedrange=True)

## Visualising hourly pedestrian traffic distribution

In [ ]:
%%sql bourke_daily_df << 
SELECT
    Year,
    Date_Time::DATE AS Date,
    sum(Hourly_Counts) AS Daily_Counts,
FROM pedestrian_counts
WHERE Sensor_Name = 'Bourke Street Mall (North)'
    AND Year IN (2019, 2020, 2021)
GROUP BY Year, Date

In [ ]:
bourke_daily_df.head()

In [ ]:
px.box(
    bourke_daily_df,
    x="Year",
    y="Daily_Counts",
    points="all",
    height=500,
    title="Distribution of daily total traffic for Bourke Street Mall (North)",
)

## Summary